# Generate FORTRAN static modules with OTI or Multidual algebras.
```
Author: Mauricio Aristizabal
Date last modified: Jan 17 2022
Initial date: Jan 13 2022
Contact: mauriaristi@gmail.com
```

This Notebook lets you generate Fortran modules that implements static versions of the library.

They implement the so called semi-sparse implementation to avoid computational overheads.


In [1]:
import pyoti.core   as coti
from pyoti.fmod_writer import writer

In [10]:
nbases = 3
order  = 3
w = writer(nbases, order,mdual=True)

In [11]:

print(w.multiplication_like_function_oo( level = "", f_name = "", lhs_name= "LHS", rhs_name= "RHS", 
       res_name = "RES", separator = "*", f_open = "", f_close = "",  addition = " + "))

!  Multiplication like function 'LHS*RHS'
! Order 0
RES%R = LHS%R*RHS%R
! Order 1
RES%E1 = LHS%R*RHS%E1 + LHS%E1*RHS%R
RES%E2 = LHS%R*RHS%E2 + LHS%E2*RHS%R
RES%E3 = LHS%R*RHS%E3 + LHS%E3*RHS%R
! Order 2
RES%E12 = LHS%R*RHS%E12 + LHS%E12*RHS%R + LHS%E1*RHS%E2 &
        + LHS%E2*RHS%E1
RES%E13 = LHS%R*RHS%E13 + LHS%E13*RHS%R + LHS%E1*RHS%E3 &
        + LHS%E3*RHS%E1
RES%E23 = LHS%R*RHS%E23 + LHS%E23*RHS%R + LHS%E2*RHS%E3 &
        + LHS%E3*RHS%E2
! Order 3
RES%E123 = LHS%R*RHS%E123 + LHS%E123*RHS%R + LHS%E1*RHS%E23 &
         + LHS%E23*RHS%E1 + LHS%E2*RHS%E13 + LHS%E13*RHS%E2 &
         + LHS%E3*RHS%E12 + LHS%E12*RHS%E3



In [13]:
w.mult_res

[[],
 [[], []],
 [[['E1', 'E1']], [['E1', 'E2'], ['E2', 'E1']], [['E2', 'E2']]],
 [[['E1', 'E11'], ['E11', 'E1']],
  [['E1', 'E12'], ['E12', 'E1'], ['E2', 'E11'], ['E11', 'E2']],
  [['E1', 'E22'], ['E22', 'E1'], ['E2', 'E12'], ['E12', 'E2']],
  [['E2', 'E22'], ['E22', 'E2']]]]

In [21]:
w.mult_res_total[0][0]

[[[0, 0, 'R'], [0, 0, 'R']]]

In [6]:
w.name_imdir

[['0'], ['E1', 'E2', 'E3'], ['E12', 'E13', 'E23'], ['E123']]

In [7]:
print(w.addition_like_oo(level = "", operator = " + ", lhs_name= "LHS", rhs_name= "RHS", res_name = "RES"))

!  Addition like operation ' + '
!  Real
RES%R = LHS%R + RHS%R
! Order 1
RES%E1 = LHS%E1 + RHS%E1
RES%E2 = LHS%E2 + RHS%E2
RES%E3 = LHS%E3 + RHS%E3
! Order 2
RES%E12 = LHS%E12 + RHS%E12
RES%E13 = LHS%E13 + RHS%E13
RES%E23 = LHS%E23 + RHS%E23
! Order 3
RES%E123 = LHS%E123 + RHS%E123



In [12]:
w.write_file(filename = None, tab = '  ', is_std_matmul=True)

In [20]:
w.name_imdir

[['R'], ['E1', 'E2'], ['E11', 'E12', 'E22'], ['E111', 'E112', 'E122', 'E222']]